<a href="https://colab.research.google.com/github/bl00dl4nah/BD2/blob/main/Exerc%C3%ADcios%20View%2C%20Procedure%2C%20Function%20e%20Trigger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import sqlite3
import pandas as pd

# Conectar ao SQLite em memória
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Criar todas as tabelas do exercício
cursor.executescript('''
CREATE TABLE Taxista (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Nome TEXT NOT NULL,
    Salario_base REAL NOT NULL,
    Taxa_comissao REAL NOT NULL,
    Registro TEXT UNIQUE NOT NULL,
    Contato TEXT
);

CREATE TABLE Regiao (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Nome TEXT NOT NULL,
    total_taxistas INTEGER DEFAULT 0
);

CREATE TABLE Municipio (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Nome TEXT NOT NULL
);

CREATE TABLE Atende (
    ID_Taxista INTEGER,
    ID_Regiao INTEGER,
    PRIMARY KEY (ID_Taxista, ID_Regiao),
    FOREIGN KEY (ID_Taxista) REFERENCES Taxista(ID),
    FOREIGN KEY (ID_Regiao) REFERENCES Regiao(ID)
);

CREATE TABLE Regiao_Municipio (
    ID_Regiao INTEGER,
    ID_Municipio INTEGER,
    PRIMARY KEY (ID_Regiao, ID_Municipio),
    FOREIGN KEY (ID_Regiao) REFERENCES Regiao(ID),
    FOREIGN KEY (ID_Municipio) REFERENCES Municipio(ID)
);
''')

# Inserir dados de exemplo
cursor.executescript('''
INSERT INTO Taxista (Nome, Salario_base, Taxa_comissao, Registro) VALUES
('Aline Silva', 2000.00, 5.00, 'REG001'),
('João Souza', 1800.00, 4.50, 'REG002'),
('Maria Santos', 2200.00, 6.00, 'REG003');

INSERT INTO Regiao (Nome) VALUES
('Centro'),
('Zona Norte'),
('Zona Sul');

INSERT INTO Municipio (Nome) VALUES
('Belo Horizonte'),
('Contagem'),
('Betim');

INSERT INTO Regiao_Municipio (ID_Regiao, ID_Municipio) VALUES
(1, 1), (2, 2), (3, 3);

INSERT INTO Atende (ID_Taxista, ID_Regiao) VALUES
(1, 1), (1, 2), (2, 3), (3, 1);
''')

conn.commit()

# Testar uma consulta
df = pd.read_sql_query("SELECT * FROM Taxista", conn)
print("Dados da tabela Taxista:")
print(df)

Dados da tabela Taxista:
   ID          Nome  Salario_base  Taxa_comissao Registro Contato
0   1   Aline Silva        2000.0            5.0   REG001    None
1   2    João Souza        1800.0            4.5   REG002    None
2   3  Maria Santos        2200.0            6.0   REG003    None


In [20]:
print("Total de taxistas por região (apenas chaves estrangeiras) ---")
sql_q1 = """
SELECT
    ID_Regiao,
    COUNT(ID_Taxista) as total_taxistas
FROM Atende
GROUP BY ID_Regiao;
"""
df_q1 = pd.read_sql_query(sql_q1, conn)
print(df_q1)

Total de taxistas por região (apenas chaves estrangeiras) ---
   ID_Regiao  total_taxistas
0          1               2
1          2               1
2          3               1


In [21]:
try:
    cursor.execute("DROP VIEW IF EXISTS vw_total_taxistas_por_regiao;")
    sql_q2_view = """
    CREATE VIEW vw_total_taxistas_por_regiao AS
    SELECT
        T2.Nome AS Nome_Regiao,
        COUNT(T1.ID_Taxista) AS total_taxistas
    FROM Atende AS T1
    JOIN Regiao AS T2 ON T1.ID_Regiao = T2.ID
    GROUP BY T1.ID_Regiao;
    """
    cursor.execute(sql_q2_view)

    sql_q2_select = "SELECT * FROM vw_total_taxistas_por_regiao;"
    df_q2 = pd.read_sql_query(sql_q2_select, conn)
    print(df_q2)

except sqlite3.OperationalError as e:
    print(f"Erro ao criar a VIEW: {e}. O SQLite não suporta VIEWs com GROUP BY, o que impede a execução desta e das próximas questões. As questões seguintes serão implementadas como consultas diretas.")

  Nome_Regiao  total_taxistas
0      Centro               2
1  Zona Norte               1
2    Zona Sul               1


In [22]:
try:
    cursor.execute("DROP VIEW IF EXISTS vw_municipios_aline_silva;")
    sql_q3_view = """
    CREATE VIEW vw_municipios_aline_silva AS
    SELECT DISTINCT
        T3.Nome AS Nome_Municipio,
        T2.Nome AS Nome_Regiao
    FROM Atende AS T1
    JOIN Regiao AS T2 ON T1.ID_Regiao = T2.ID
    JOIN Regiao_Municipio AS T4 ON T2.ID = T4.ID_Regiao
    JOIN Municipio AS T3 ON T4.ID_Municipio = T3.ID
    JOIN Taxista AS T5 ON T1.ID_Taxista = T5.ID
    WHERE T5.Nome = 'Aline Silva';
    """
    cursor.execute(sql_q3_view)

    sql_q3_select = "SELECT * FROM vw_municipios_aline_silva;"
    df_q3 = pd.read_sql_query(sql_q3_select, conn)
    print(df_q3)
except sqlite3.OperationalError as e:
    print(f"Erro ao criar a VIEW: {e}. Pulando para a próxima questão.")

   Nome_Municipio Nome_Regiao
0  Belo Horizonte      Centro
1        Contagem  Zona Norte


In [23]:
def procedure_consulta_municipios(conn, taxista_nome):
    print(f"Chamando PROCEDURE para o taxista '{taxista_nome}'...")
    sql_proc = f"""
    SELECT DISTINCT
        T3.Nome AS Nome_Municipio,
        T2.Nome AS Nome_Regiao
    FROM Atende AS T1
    JOIN Regiao AS T2 ON T1.ID_Regiao = T2.ID
    JOIN Regiao_Municipio AS T4 ON T2.ID = T4.ID_Regiao
    JOIN Municipio AS T3 ON T4.ID_Municipio = T3.ID
    JOIN Taxista AS T5 ON T1.ID_Taxista = T5.ID
    WHERE T5.Nome = ?;
    """
    df_proc = pd.read_sql_query(sql_proc, conn, params=(taxista_nome,))
    print(df_proc)

procedure_consulta_municipios(conn, 'João Souza')

Chamando PROCEDURE para o taxista 'João Souza'...
  Nome_Municipio Nome_Regiao
0          Betim    Zona Sul


In [24]:
def function_qtd_municipios(conn, taxista_nome):
    sql_func = f"""
    SELECT
        COUNT(DISTINCT T4.ID_Municipio) AS total_municipios
    FROM Atende AS T1
    JOIN Regiao AS T2 ON T1.ID_Regiao = T2.ID
    JOIN Regiao_Municipio AS T4 ON T2.ID = T4.ID_Regiao
    JOIN Taxista AS T5 ON T1.ID_Taxista = T5.ID
    WHERE T5.Nome = ?;
    """
    cursor.execute(sql_func, (taxista_nome,))
    result = cursor.fetchone()
    return result[0] if result else 0

total_municipios_joao = function_qtd_municipios(conn, 'João Souza')
print(f"SELECT com a FUNCTION: 'João Souza' atende {total_municipios_joao} municípios.")

SELECT com a FUNCTION: 'João Souza' atende 1 municípios.


In [25]:
class TaxistaNaoCadastrada(Exception):
    def __init__(self, message="Taxista não cadastrada, verifique o nome informado"):
        self.message = message
        super().__init__(self.message)

def function_qtd_municipios_verificada(conn, taxista_nome):
    cursor.execute("SELECT COUNT(*) FROM Taxista WHERE Nome = ?", (taxista_nome,))
    if cursor.fetchone()[0] == 0:
        raise TaxistaNaoCadastrada()

    sql_func = f"""
    SELECT
        COUNT(DISTINCT T4.ID_Municipio) AS total_municipios
    FROM Atende AS T1
    JOIN Regiao AS T2 ON T1.ID_Regiao = T2.ID
    JOIN Regiao_Municipio AS T4 ON T2.ID = T4.ID_Regiao
    JOIN Taxista AS T5 ON T1.ID_Taxista = T5.ID
    WHERE T5.Nome = ?;
    """
    cursor.execute(sql_func, (taxista_nome,))
    return cursor.fetchone()[0]

try:
    total_municipios_maria = function_qtd_municipios_verificada(conn, 'Maria Santos')
    print(f"SELECT com a FUNCTION: 'Maria Santos' atende {total_municipios_maria} municípios.")
    total_municipios_nao_existe = function_qtd_municipios_verificada(conn, 'José da Silva')
except TaxistaNaoCadastrada as e:
    print(f"Erro customizado: {e}")


SELECT com a FUNCTION: 'Maria Santos' atende 1 municípios.
Erro customizado: Taxista não cadastrada, verifique o nome informado


In [26]:
try:
    cursor.execute("DROP TRIGGER IF EXISTS trg_aumenta_salario;")
    sql_q7_trigger = """
    CREATE TRIGGER trg_aumenta_salario
    AFTER INSERT ON Atende
    FOR EACH ROW
    BEGIN
        UPDATE Taxista
        SET Salario_base = Salario_base * 1.1
        WHERE ID = NEW.ID_Taxista;
    END;
    """
    cursor.execute(sql_q7_trigger)

    print("Salários antes da inserção:")
    df_salarios_antes = pd.read_sql_query("SELECT Nome, Salario_base FROM Taxista;", conn)
    print(df_salarios_antes)

    print("\nExecutando o evento que dispara o gatilho...")
    cursor.execute("INSERT INTO Atende (ID_Taxista, ID_Regiao) VALUES (2, 2);")
    conn.commit()

    print("\nSalários após a inserção (salário de João Souza deve aumentar):")
    df_salarios_depois = pd.read_sql_query("SELECT Nome, Salario_base FROM Taxista;", conn)
    print(df_salarios_depois)

except sqlite3.OperationalError as e:
    print(f"Erro ao criar o TRIGGER: {e}. Pulando para a próxima questão.")

Salários antes da inserção:
           Nome  Salario_base
0   Aline Silva        2000.0
1    João Souza        1800.0
2  Maria Santos        2200.0

Executando o evento que dispara o gatilho...

Salários após a inserção (salário de João Souza deve aumentar):
           Nome  Salario_base
0   Aline Silva        2000.0
1    João Souza        1980.0
2  Maria Santos        2200.0


In [27]:
try:
    cursor.execute("DROP TRIGGER IF EXISTS trg_diminui_salario;")
    sql_q8_trigger = """
    CREATE TRIGGER trg_diminui_salario
    AFTER DELETE ON Atende
    FOR EACH ROW
    BEGIN
        UPDATE Taxista
        SET Salario_base = Salario_base / 1.1
        WHERE ID = OLD.ID_Taxista;
    END;
    """
    cursor.execute(sql_q8_trigger)

    print("Salários antes da exclusão:")
    df_salarios_antes_del = pd.read_sql_query("SELECT Nome, Salario_base FROM Taxista;", conn)
    print(df_salarios_antes_del)

    print("\nExecutando o evento que dispara o gatilho...")
    cursor.execute("DELETE FROM Atende WHERE ID_Taxista = 2 AND ID_Regiao = 3;")
    conn.commit()

    print("\nSalários após a exclusão (salário de João Souza deve diminuir):")
    df_salarios_depois_del = pd.read_sql_query("SELECT Nome, Salario_base FROM Taxista;", conn)
    print(df_salarios_depois_del)

except sqlite3.OperationalError as e:
    print(f"Erro ao criar o TRIGGER: {e}. A lógica do código é correta, mas a execução pode falhar dependendo da versão do SQLite.")

conn.close()


Salários antes da exclusão:
           Nome  Salario_base
0   Aline Silva        2000.0
1    João Souza        1980.0
2  Maria Santos        2200.0

Executando o evento que dispara o gatilho...

Salários após a exclusão (salário de João Souza deve diminuir):
           Nome  Salario_base
0   Aline Silva        2000.0
1    João Souza        1800.0
2  Maria Santos        2200.0
